In [10]:
using CSV
using DataFrames
using Dates
using Statistics
using LinearAlgebra
using JuMP
using Gurobi

data_dir = "./data"

# ─── HELPERS ───────────────────────────────────────────────────────────────────

"""
    compute_returns(prices::Vector{Float64})

From a sorted price series returns the vector of simple returns r_t = (P_t/P_{t-1}) - 1
"""
function compute_returns(prices::Vector{Float64})
    diff(prices) ./ prices[1:end-1]
end

# ─── LOAD, COMPUTE RETURNS & LIQUIDITY ────────────────────────────────────────

# We'll build two parallel lists of DataFrames:
#  - dfs_returns[i] has columns: Date, r_i
#  - liquidity[i]    is mean(Volume) for that same asset (aligned to returns)

dfs_returns = DataFrame[]
liquidity   = Float64[]

for file in filter(f->endswith(lowercase(f), ".csv"), readdir(data_dir; join=true))
    df = CSV.read(file, DataFrame)

    # 1) parse & sort
    df.Date = Date.(df.Date, dateformat"u d, Y")  # e.g. "May 7, 2025"
    sort!(df, :Date)

    # 2) returns r_t for t=2:end
    rs = compute_returns(df.Close)

    # 3) volume aligned to returns is df.Volume[2:end]
    vs = df.Volume[2:end]

    # 4) drop row-1 since no return there
    sub = df[2:end, [:Date]]
    sub.:return = rs
    # name the return-col later

    push!(dfs_returns, sub)
    # compute average weekly volume as a liquidity metric
    push!(liquidity, mean(vs))
end

# rename each return column to r1, r2, …
for (i, df) in enumerate(dfs_returns)
    rename!(df, :return => Symbol("r$i"))
end

# join all returns on Date
rets = reduce((a,b)->innerjoin(a,b,on=:Date), dfs_returns)

# ─── COMPUTE μ and Σ ──────────────────────────────────────────────────────────

# expected weekly returns
μ = [ mean(rets[!, Symbol("r$i")]) for i in 1:length(dfs_returns) ]

# covariance of returns
R = Matrix(rets[:, Not(:Date)])
Σ = Symmetric(cov(R, dims=1))

println("μ = ", μ)
println("Σ = "); display(Σ)

# ─── LIQUIDITY VECTOR ─────────────────────────────────────────────────────────

println("Average weekly volumes (liquidity) = ", liquidity)


μ = [0.002592239832991949, 0.004611705992668354, 0.0014443258538598808]
Σ = 


3×3 Symmetric{Float64, Matrix{Float64}}:
 0.0035613    0.00139997   0.000828436
 0.00139997   0.00197079   0.000601053
 0.000828436  0.000601053  0.00124855

Average weekly volumes (liquidity) = [4.305632115384615e7, 1.126198076923077e7, 1.0168295384615384e8]


In [13]:
using JuMP, Gurobi
using LinearAlgebra

"""
    objective(w, w_prev, Σ, μ, γ, L, λ)

Compute

    lamda wᵀ sigma w  -  (1-lamda) wᵀ mu  +  ∑ᵢ gamma[i] * |w[i] - w_prev[i]| / L[i]

Inputs
- `w`       : current weight vector, size n
- `w_prev`  : previous weight vector, size n
- `sigma`       : n×n covariance matrix
- `mu`       : n-vector of expected returns
- `gamma`       : n-vector of sparsity penalties
- `L`       : n-vector of scaling factors
- `lamda`       : scalar trade‐off parameter in [0,1]
"""

# — Example usage of objective() directly —
n = 3
mu = μ
sigma      =  Σ   # example covariance
gamma      = fill(100, n)
L      = liquidity          # ensure nonzero
w_prev = fill(1/n, n)  # previous weights
w      = randn(n)
lamda      = 0.5

model = Model(Gurobi.Optimizer)
@variable(model, w[1:n])
@variable(model, z[1:n] >= 0)  # auxiliary variable for absolute value
# (you can add constraints here, e.g. sum(w) == 1, w .>= 0, etc.)
@constraint(model, sum(w) == 1)
@constraint(model, w .>= 0)
@constraint(model, w .<= 1)
@constraint(model, z .>= w - w_prev)
@constraint(model, z .>= -(w - w_prev))
@objective(model, Min,
    lamda * w' * sigma * w
  - (1 - lamda) * dot(w, μ)
  + sum( gamma[i] * z[i] / L[i] for i in 1:n )
)
optimize!(model)
println("\nObjective value: ", objective_value(model))
println("Optimal weights: ", value.(w))

Set parameter Username
Set parameter LicenseID to value 2650745
Academic license - for non-commercial use only - expires 2026-04-12
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 13 rows, 6 columns and 21 nonzeros
Model fingerprint: 0x942a3097
Model has 6 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 2e-03]
  QObjective range [1e-03, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+00]
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 6 columns, 15 nonzeros
Presolved model has 6 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 2
 AA' NZ     : 2.200e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective  